# **Partition a New Plane:**

Problem Source: Birge, John R., and Francois Louveaux. Introduction to stochastic programming. Springer Science & Business Media, 2011.

**Problem Statement:**

Northam Airlines is trying to decide how to partition a new plane for its Chicago–Detroit route. The plane can seat $200$ economy class passengers. A section can be partitioned off for first class seats but each of these seats takes the space of $2$ economy class seats. A business class section can also be included, but each of these seats takes as much space as $1.5$ economy class seats. The profit on a first class ticket is, however, three times the profit of an economy
ticket. A business class ticket has a profit of two times an economy ticket’s profit. Once the plane is partitioned into these seating classes, it cannot be changed. Northam knows, however, that the plane will not always be full in each section. They have decided that three scenarios will occur with about the same frequency: $(1)$ weekday morning and evening traffic, $(2)$ weekend traffic, and $(3)$ weekday midday traffic. Under Scenario $1$, they think they can sell as many as $20$ first class tickets, $50$ business class tickets, and $200$ economy tickets. Under Scenario $2$, these figures are $10$, $25$, and $175$ . Under Scenario $3$, they are $5$, $10$, and $150$ . You can assume they cannot sell more tickets than seats in each of the sections. (In reality, the company may allow overbooking, but then it faces the problem of passengers with reservations who do not appear for the flight (no-shows). The problem of determining how many passengers to accept is part of the field called yield management or revenue management. For one approach to this problem, see Brumelle and McGill [1993].

**Modeling:**

$\\ $

$variables:\\ $

$s$: Scenarios

$c$: Three different classes  $\rightarrow$ economy, business, first class 

$x_{c}$: The number of seats allocated for class $c$

$w_{sc}$: Number of class $c$ seats under scenario $s$

$p_{s}$: probability of scenario $s$

$d_{sc}$: demand for class $c$ seats under scenario $s$

$\\ $

$Objective\ Function:\\ $

$Max\  p_{1} \big(3w_{11} + 3w_{21} + 3w_{31}\big) + p_{2}  \big(3w_{12} + 3w_{22} + 3w_{32}\big) + p_{3}  \big(3w_{13} + 3w_{23} + 3w_{33}\big)$

$\\ $

$Constraints:\\ $

$ \sum_1^c x_{c} = 200 $

$w_{sc}  \leq d_{sc}$

$w_{sc}  \leq x_{c}$

$w, X  \in integer$

In [1]:
!pip install pulp
from pulp import *
import numpy as np
import pandas as pd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
#Defining the problem

Total_seats = 200
No_scenarios = 3
s11 = list(range(1,No_scenarios+1))
s22 = list(range(1,No_scenarios+1))
s33 = list(range(1,No_scenarios+1))
s_1 = [20 ,50, 200]
s_2 = [10, 25, 175]
s_3 = [5, 10, 150]
profit = [2, 1.5, 1]
p = [1/3, 1/3, 1/3]

In [3]:
#Defining variables

X = LpVariable.dicts('X',((s) for s in s11), lowBound=0, cat='Integer')
w = LpVariable.dicts('w',((s1,s2) for s1 in s11 for s2 in s22), lowBound=0, cat='Integer')

In [4]:
#Defining Objective Function

Model= LpProblem('Maximize_Profit', LpMaximize)

Model+= p[0]*lpSum(3*w[1,s1] for s1 in s11) + p[1]*lpSum(2*w[2,s2] for s2 in s22) + p[2]*lpSum(1*w[3,s3] for s3 in s33)

In [5]:
#Constraints

Model+= lpSum(X[s1]*profit[s1-1] for s1 in s11) == Total_seats

for s1 in s11:
    Model+= w[s1,1] <= s_1[s1-1]

for s2 in s22:
    Model+= w[s2,2] <= s_2[s2-1]

for s3 in s33:
    Model+= w[s3,3] <= s_3[s3-1]

for s1 in s11:
    Model+= w[s1,1] <= X[s1]

for s2 in s22:
    Model+= w[s2,2] <= X[s2]

for s3 in s33:
    Model+= w[s3,3] <= X[s3]

In [6]:
#Model summary

Model

Maximize_Profit:
MAXIMIZE
1.0*w_(1,_1) + 1.0*w_(1,_2) + 1.0*w_(1,_3) + 0.6666666666666666*w_(2,_1) + 0.6666666666666666*w_(2,_2) + 0.6666666666666666*w_(2,_3) + 0.3333333333333333*w_(3,_1) + 0.3333333333333333*w_(3,_2) + 0.3333333333333333*w_(3,_3) + 0.0
SUBJECT TO
_C1: 2 X_1 + 1.5 X_2 + X_3 = 200

_C2: w_(1,_1) <= 20

_C3: w_(2,_1) <= 50

_C4: w_(3,_1) <= 200

_C5: w_(1,_2) <= 10

_C6: w_(2,_2) <= 25

_C7: w_(3,_2) <= 175

_C8: w_(1,_3) <= 5

_C9: w_(2,_3) <= 10

_C10: w_(3,_3) <= 150

_C11: - X_1 + w_(1,_1) <= 0

_C12: - X_2 + w_(2,_1) <= 0

_C13: - X_3 + w_(3,_1) <= 0

_C14: - X_1 + w_(1,_2) <= 0

_C15: - X_2 + w_(2,_2) <= 0

_C16: - X_3 + w_(3,_2) <= 0

_C17: - X_1 + w_(1,_3) <= 0

_C18: - X_2 + w_(2,_3) <= 0

_C19: - X_3 + w_(3,_3) <= 0

VARIABLES
0 <= X_1 Integer
0 <= X_2 Integer
0 <= X_3 Integer
0 <= w_(1,_1) Integer
0 <= w_(1,_2) Integer
0 <= w_(1,_3) Integer
0 <= w_(2,_1) Integer
0 <= w_(2,_2) Integer
0 <= w_(2,_3) Integer
0 <= w_(3,_1) Integer
0 <= w_(3,_2) Integer
0 <= w_(3,

In [7]:
#Solving model

Model.solve()
print('Optimal Solution:', pulp.value(Model.objective))

Optimal Solution: 208.33333333333331


In [8]:
#Optimal values of variables

for variables in Model.variables():
  print('{}:{}'.format(variables.name, variables.varValue))

X_1:10.0
X_2:20.0
X_3:150.0
w_(1,_1):10.0
w_(1,_2):10.0
w_(1,_3):5.0
w_(2,_1):20.0
w_(2,_2):20.0
w_(2,_3):10.0
w_(3,_1):150.0
w_(3,_2):150.0
w_(3,_3):150.0


In [9]:
VNames = []
for variables in Model.variables():
  VNames.append(variables.name)

VValue = []
for variables in Model.variables():
  VValue.append(np.round(variables.varValue, 2))

data = {'Variables': VNames, 'Value': VValue}
pd.DataFrame(data, index = range(1, len(VValue)+1))

,Variables,Value
1,X_1,10.0
2,X_2,20.0
3,X_3,150.0
4,"w_(1,_1)",10.0
5,"w_(1,_2)",10.0
6,"w_(1,_3)",5.0
7,"w_(2,_1)",20.0
8,"w_(2,_2)",20.0
9,"w_(2,_3)",10.0
10,"w_(3,_1)",150.0


In [10]:
print('Current Status: ', LpStatus[Model.status])

Current Status:  Optimal
